In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numba as nb

### Age profile of income 

In [2]:
## import age income profile 

age_profile = pd.read_stata('../OtherData/age_profile.dta')   

## select age range for the model and turn it into an array 
lc_wages = np.array(age_profile[(age_profile['age']>=25) &(age_profile['age']<=64)]['wage_age'])
#print(str(len(lc_wages)),'years since age 25')

## growth rates since initial age over life cicle before retirement
lc_G = lc_wages[1:]/lc_wages[:-1]
lc_G = np.insert(lc_G,0,1.0) 

T = 40
L = 60
## growth rates after retirement

lc_G_rt = np.ones(L-T)
lc_G_rt[1] = 0.6


lc_G_full = np.concatenate([lc_G,lc_G_rt])
#lc_G_full = np.ones_like(lc_G_full)

T_q = T*4
L_q = L*4

In [3]:
## turn yearly number to quarterly number with interpolation 

def y2q_interpolate(xs_y):
    
    """
    this function turns an array of yealry rates into quarterly rates via linear interpolation 
    """

    n_y = len(xs_y)
    y_id = np.arange(n_y)
    q_id = y_id*4

    n_q = 4*n_y
    xs_q = np.empty(n_q)
    
    for i_y in range(n_y):
        xs_q[4*i_y] = xs_y[i_y]
        for i_q in np.arange(1,4):
            q_id_this = 4*i_y+i_q
            xs_q[q_id_this] = np.interp(q_id_this,q_id,xs_y)
            
    return xs_q


## get the quarterly income profile over life cycle before retirement 
lc_G_q = y2q_interpolate(lc_G)

lc_G_q_rt = y2q_interpolate(lc_G_rt)

lc_G_q_full = np.concatenate([lc_G_q,lc_G_q_rt])

### Income risk estimates 

In [4]:
risks_est = pd.read_stata('../OtherData/sipp/sipp_history_vol_decomposed.dta')
## risks of permanent and transitory component 

σ_ψ_q_sipp = np.sqrt(risks_est['permanent']**2*3)
σ_θ_q_sipp = np.sqrt(risks_est['permanent']**2/3)

## p/t ratio 
kappas_sipp  = risks_est['permanent']/risks_est['transitory']
kappa_sipp = np.median(kappas_sipp.dropna())

In [5]:
## risks of permanent and transitory component 

σ_ψ_q_sipp = np.sqrt(risks_est['permanent']**2*3)
σ_θ_q_sipp = np.sqrt(risks_est['permanent']**2/3)

## p/t ratio 
kappas_sipp  = risks_est['permanent']/risks_est['transitory']
kappa_sipp = np.median(kappas_sipp.dropna())

### Initial conditions

In [6]:
from statsmodels.stats.weightstats import DescrStatsW

## SCF data 
SCF2016 = pd.read_stata('rscfp2016.dta')
SCF2016 = SCF2016[SCF2016['age']==25]
SCF2016 = SCF2016.drop_duplicates(subset=['yy1'])
SCF2016 = SCF2016[['norminc','networth','wgt']]
SCF2016 = SCF2016[SCF2016['norminc']>0]

## permanent income at age 25 
pinc_SCF = SCF2016[['norminc','wgt']]
lpinc_SCF_pos, lpinc_SCF_wgt = np.log(pinc_SCF['norminc']),np.array(pinc_SCF['wgt'])

## wealth 
b_SCF = SCF2016[['networth','wgt']]
lb_SCF_pos, lb_SCF_wgt = b_SCF['networth'],np.array(b_SCF['wgt'])

## wealth to permanent income
SCF2016['networth2pinc'] = SCF2016['networth']/SCF2016['norminc']
b_SCF, b_wgt = SCF2016['networth2pinc'], SCF2016['wgt']

## compute data moments 
σ_ψ_init_SCF = DescrStatsW(lpinc_SCF_pos, weights=lpinc_SCF_wgt, ddof=1).std
σ_b_init_SCF = DescrStatsW(lb_SCF_pos, weights=lb_SCF_wgt, ddof=1).std
b_SCF = DescrStatsW(b_SCF,weights=b_wgt, ddof=1).mean ## quarterly 
b_q_SCF = b_SCF*4

### subjective profile estiamtion 

In [7]:
## import subjective profile estimation results 
SCE_est_q = pd.read_pickle('data/subjective_profile_est_q.pkl')
SCE_est_y = pd.read_pickle('data/subjective_profile_est_y.pkl')
SCE_est_q = SCE_est_q['baseline']
SCE_est_y = SCE_est_y['baseline']

AttributeError: Can't get attribute 'new_block' on <module 'pandas.core.internals.blocks' from '/Users/tao/opt/anaconda3/envs/econ-ark/lib/python3.9/site-packages/pandas/core/internals/blocks.py'>

In [8]:
## create a dictionary of parameters 
life_cycle_paras_q = {'ρ': 1.0, 
                    'β': 0.98**(1/4), 
                    'P': np.array([[0.18, 0.82],
                                   [0.04, 0.96]]), 
                    'z_val': np.array([0., 1.]), 
                    'σ_ψ': np.sqrt(0.15**2*4/11), 
                    'σ_θ': np.sqrt(0.1**2*4), 
                    'U': 0.0, 
                    'LivPrb': 1.0-0.00625, 
                    'R': 1.01**(1/4), 
                    'W': 1.0, 
                    'T': T_q, 
                    'L': L_q, 
                    'G':lc_G_q_full, 
                    'unemp_insurance': 0.15, 
                    'pension': 1.0, 
                    'σ_ψ_init': σ_ψ_init_SCF, 
                    'init_b': b_q_SCF, 
                    'λ': 0.0, 
                    'λ_SS': 0.0, 
                    'transfer': 0.0, 
                    'bequest_ratio': 0.0,
                    'κ':kappa_sipp,
                    
                    ## subjective profile
                    'q':SCE_est_q.loc['$q$'],
                    'p':SCE_est_q.loc['$p$'],
                    'σ_ψ_2mkv':np.array([SCE_est_q.loc['$\tilde\sigma^l_\psi$'],
                                       SCE_est_q.loc['$\tilde\sigma^h_\psi$']]),
                    'σ_θ_2mkv':np.array([SCE_est_q.loc['$\tilde\sigma^l_\theta$'],
                                       SCE_est_q.loc['$\tilde\sigma^h_\theta$']]),
                    'mho_2mkv':np.array([SCE_est_q.loc['$\tilde \mho^l$'],
                                         SCE_est_q.loc['$\tilde \mho^h$']]),
                    'E_2mkv':np.array([SCE_est_q.loc['$\tilde E^l$'],
                                      SCE_est_q.loc['$\tilde E^h$']])
                }

NameError: name 'SCE_est_q' is not defined

In [12]:
life_cycle_paras_q

{'ρ': 1.0,
 'β': 0.9949620563926881,
 'P': array([[0.18, 0.82],
        [0.04, 0.96]]),
 'z_val': array([0., 1.]),
 'σ_ψ': 0.09045340337332908,
 'σ_θ': 0.2,
 'U': 0.0,
 'LivPrb': 0.99375,
 'R': 1.0024906793143211,
 'W': 1.0,
 'T': 160,
 'L': 240,
 'G': array([1.        , 1.00621092, 1.01242185, 1.01863277, 1.02484369,
        1.02443039, 1.0240171 , 1.0236038 , 1.0231905 , 1.02278841,
        1.02238631, 1.02198422, 1.02158213, 1.02119136, 1.02080059,
        1.02040982, 1.02001905, 1.01963976, 1.01926047, 1.01888117,
        1.01850188, 1.01813418, 1.01776648, 1.01739877, 1.01703107,
        1.01667506, 1.01631904, 1.01596302, 1.015607  , 1.01526287,
        1.01491874, 1.01457462, 1.01423049, 1.0138984 , 1.01356632,
        1.01323423, 1.01290214, 1.01258209, 1.01226205, 1.011942  ,
        1.01162195, 1.01131424, 1.01100653, 1.01069883, 1.01039112,
        1.01009578, 1.00980043, 1.00950509, 1.00920975, 1.00892699,
        1.00864422, 1.00836146, 1.00807869, 1.00780857, 1.00753844,


In [13]:
## create a dictionary of parameters 
life_cycle_paras_y = {'ρ': 1.0, 
                    'β': 0.98, 
                    'P': np.array([[0.18, 0.82],
                                   [0.04, 0.96]]), 
                    'z_val': np.array([0., 1.]), 
                    'σ_ψ': np.sqrt(0.15**2), 
                    'σ_θ': np.sqrt(0.1**2), 
                    'U': 0.0, 
                    'LivPrb': 1.0-0.00625, 
                    'R': 1.01, 
                    'W': 1.0, 
                    'T': T, 
                    'L': L, 
                    'G':lc_G_full, 
                    'unemp_insurance': 0.15, 
                    'pension': 1.0, 
                    'σ_ψ_init': σ_ψ_init_SCF, 
                    'init_b': b_SCF, 
                    'λ': 0.0, 
                    'λ_SS': 0.0, 
                    'transfer': 0.0, 
                    'bequest_ratio': 0.0,
                    'κ':kappa_sipp,
                    
                    ## subjective profile
                    'q':SCE_est_y.loc['$q$'],
                    'p':SCE_est_y.loc['$p$'],
                    'σ_ψ_2mkv':np.array([SCE_est_y.loc['$\tilde\sigma^l_\psi$'],
                                       SCE_est_y.loc['$\tilde\sigma^h_\psi$']]),
                    'σ_θ_2mkv':np.array([SCE_est_y.loc['$\tilde\sigma^l_\theta$'],
                                       SCE_est_y.loc['$\tilde\sigma^h_\theta$']]),
                    'mho_2mkv':np.array([SCE_est_y.loc['$\tilde \mho^l$'],
                                         SCE_est_y.loc['$\tilde \mho^h$']]),
                    'E_2mkv':np.array([SCE_est_y.loc['$\tilde E^l$'],
                                      SCE_est_y.loc['$\tilde E^h$']])
                }

In [14]:
life_cycle_paras_y

{'ρ': 1.0,
 'β': 0.98,
 'P': array([[0.18, 0.82],
        [0.04, 0.96]]),
 'z_val': array([0., 1.]),
 'σ_ψ': 0.15,
 'σ_θ': 0.1,
 'U': 0.0,
 'LivPrb': 0.99375,
 'R': 1.01,
 'W': 1.0,
 'T': 40,
 'L': 60,
 'G': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'unemp_insurance': 0.15,
 'pension': 1.0,
 'σ_ψ_init': 0.6285209184601319,
 'init_b': 0.5540657379463797,
 'λ': 0.0,
 'λ_SS': 0.0,
 'transfer': 0.0,
 'bequest_ratio': 0.0,
 'κ': 1.719786711116489,
 'q': 0.28170847347106975,
 'p': 0.28170847347106975,
 'σ_ψ_2mkv': array([0.44911376, 0.93665   ]),
 'σ_θ_2mkv': array([0.01088104, 0.02269298]),
 'mho_2mkv': array([-0.39612487,  1.73622263]),
 'E_2mkv': array([0.4364263 , 0.93597574])}

In [15]:
#with open("parameters.txt", "wb") as fp:
#    pickle.dump(life_cycle_paras, fp)